# VacationsPy using Google Maps API / Heatmaps and Markers

In [1]:
%matplotlib notebook

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

## Store part1 results into a dataframe

In [3]:
#Read the cities.csv file created in WeatherPy.ipynp
cities_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities_df

,Unnamed: 0,Latitude,Temperature,City,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,42.60,83.88,Albany,-73.97,86.00,43,33,9.10,US,1595352623
1,1,-54.80,35.60,Ushuaia,-68.30,35.60,93,60,2.24,AR,1595352831
2,2,63.92,56.91,Sangar,127.47,56.91,95,100,15.84,RU,1595352971
3,3,-34.48,68.38,Rocha,-54.33,68.38,87,100,4.05,UY,1595353134
4,4,-23.12,66.13,Rikitea,-134.97,66.13,60,46,17.94,PF,1595352861
...,...,...,...,...,...,...,...,...,...,...,...
553,553,36.75,87.80,Algiers,3.04,87.80,58,0,9.17,DZ,1595353250
554,554,-3.37,75.88,Nabire,135.48,75.88,86,96,4.00,ID,1595353251
555,555,25.52,73.98,Tura,90.22,73.98,98,98,5.28,IN,1595352870
556,556,58.70,57.54,Parabel',81.48,57.54,77,47,10.00,RU,1595353253


## Humidity Heatmap

In [4]:
#Convert the datatype of Humidity column as float
humidity = cities_df["Humidity"].astype(float)
maxhumidity = humidity.max()
#Create a DataFrame to hold the latitudes and longitudes
locations = cities_df[["Latitude", "Longitude"]]

In [5]:
#Load Google Maps
cent =(0,0)

fig = gmaps.figure(center=cent,zoom_level =2.0)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria

In [6]:
#Filter a DataFrame for locations where max temperature  is >70 & <80 & cloudiness ==0
new_df = cities_df.loc[(cities_df["Max Temperature"] > 70) & (cities_df["Max Temperature"] < 80) & (cities_df["Cloudiness"] == 0), :]
#Drop any ___ rows
new_df = new_df.dropna(how='any')
new_df.reset_index(inplace=True)
del new_df['index']
new_df.head()

,Unnamed: 0,Latitude,Temperature,City,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,9,-24.79,71.89,Castro,-50.01,71.89,53,0,7.18,BR,1595352841
1,229,45.76,71.29,Mandalgovi,106.27,71.29,24,0,5.14,MN,1595353187
2,253,39.70,72.21,Linxi,118.43,72.21,70,0,3.38,CN,1595353196
3,324,31.75,75.94,Taft,54.21,75.94,18,0,2.64,IR,1595353213
4,505,43.48,72.93,Ferrol,-8.24,75.20,60,0,10.29,ES,1595353243


In [7]:
hotellist = []
#Iterate through the list of locations and obtain nearest hotel using google API
for i in range(len(new_df)):
    lat = new_df.loc[i]['Latitude']
    lng = new_df.loc[i]['Longitude']
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
#Add Hotel Name to the DataFrame
new_df["Hotel Name"] = hotellist
new_df = new_df.dropna(how='any')
new_df.head()

,Unnamed: 0,Latitude,Temperature,City,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,9,-24.79,71.89,Castro,-50.01,71.89,53,0,7.18,BR,1595352841,Castro
1,229,45.76,71.29,Mandalgovi,106.27,71.29,24,0,5.14,MN,1595353187,Mandalgovi
2,253,39.70,72.21,Linxi,118.43,72.21,70,0,3.38,CN,1595353196,Tangshan
3,324,31.75,75.94,Taft,54.21,75.94,18,0,2.64,IR,1595353213,Taft
4,505,43.48,72.93,Ferrol,-8.24,75.20,60,0,10.29,ES,1595353243,Ferrol


## Hotel Map

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in new_df.iterrows()]
locations = new_df[["Latitude", "Longitude"]]

In [9]:
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))